#### Subsequence Function

In [1]:
def subseq(seq):
    n = len(seq)
    for i in range(1, 2**n):
        b = format(i, f'0{n}b')
        s = []
        for j in range(len(b)):
            if int(b[-j-1]) == 1:
                s.append(seq[j])
        s.sort()
        yield s

### V Function

In [2]:
def v_function(A, C_values):
    """
    Computes worth of one coalition of channels
    :param A: a coalition of channels
    :param C_values: dict with conversion # of each set of channels
    :return: worth of A
    """
    worth_of_A = 0
    for subset in subseq(A.split(',')):
        subset.sort()
        subset = ','.join(subset)
        if subset in C_values.keys():
            worth_of_A += C_values[subset]
    return worth_of_A

### Import dataset

In [3]:
import pandas as pd

df = pd.read_csv('shapley_source.csv')
C_values = df.set_index('copy_list').to_dict()['conversions']
channels = sorted([c for c in C_values.keys() if ',' not in c])
print(sum(C_values.values()))

146552


### Generate V Values

In [ ]:
def get_v_values(channels, C_values):
    import os.path
    import pickle
    filename = 'v_values.pickle'
    if os.path.isfile(filename):
        with open(filename, 'rb') as file:
            return pickle.load(file)
    v_values = {}
    count = 0
    for A in subseq(channels):
        count += 1
        A = ','.join(A)
        print(f'{count}: {A}\r', end='')
        v_values[A] = v_function(A, C_values)
    print(f'\n{v_values}')
    with open(filename, 'wb') as file:
        pickle.dump(v_values, file)
    return v_values
    
v_values = get_v_values(channels, C_values)
print(v_values)

In [ ]:
print(v_values['dx,fn,vl'])

In [ ]:
def shapley(channels, v_values):
    from collections import defaultdict
    from math import factorial
    n = len(channels)
    res = defaultdict(float)
    count = 0
    for channel in channels:
        count += 1
        print(f'channel {count} of {n}')
        for A in v_values.keys():
            A_arr = A.split(',')
            if channel not in A_arr:
                cardinal_A = len(A_arr)
                A_with_channel = A_arr
                A_with_channel.append(channel)
                A_with_channel = ','.join(sorted(A_with_channel))
                res[channel] += (v_values[A_with_channel] - v_values[A])*(factorial(cardinal_A)*factorial(n-cardinal_A-1)/factorial(n))
        res[channel] += v_values[channel] / n
    return res

s = shapley(channels, v_values)

In [ ]:
for c in channels:
    assert(s[c] >= C_values[c])
from pprint import pprint
pprint(s)
print(sum(s.values()))

### Plot Result Histogram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(30, 10))
ax = fig.add_subplot(111)
ax.bar(s.keys(), s.values(), color='r')
plt.title('Conversions by Shapley Value')
plt.xlabel('Copy')
plt.ylabel('Conversions')
plt.savefig('conversions.png', format='png')
plt.show()